In [163]:
from subprocess import Popen, PIPE
from sys import stdout, stderr
import os
import time
import dill as pickle
import json
import numpy as np
from utils import *

fw = open("tmpout", "wb")
fr = open("tmpout", "r")
p = Popen(["/Users/tianlins/Library/Android/sdk/platform-tools/adb", "shell"], stdin = PIPE, stdout = fw, stderr = fw, bufsize = 1, )



In [90]:
def block_on_result():
    while True:
        try:
            os.remove('result.txt')
        except:
            pass
        os.system('adb pull /sdcard/blas/result.txt ./')
        stdout.flush()
        if os.path.exists('result.txt'):
            with open('result.txt', 'r') as f:
                return f.read()
        time.sleep(1)
        
def test(align, **args):
    sa = args['sa']
    sb = args['sb']
    sc = args['sc']
    args['sa'] = int(np.ceil(args['sa'] / float(align)) * align)
    args['sb'] = int(np.ceil(args['sb'] / float(align)) * align)
    args['sc'] = int(np.ceil(args['sc'] / float(align)) * align)
    os.system('adb shell rm /sdcard/blas/result.txt')
    arg_str = ' '.join(['--' + key + " " + str(val) for (key, val) in args.items()]);
    p.stdin.write('''
    su
    cd /sdcard/blas
    export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:./
    ls
    ./gemm %s --output "result.txt"
    ''' % arg_str)
    result = block_on_result()
    try:
        result = json.loads(result)
    except ValueError:
        print 'json decoding error: ', result
    
    result['params']['sa'] = sa
    result['params']['sb'] = sb
    result['params']['sc'] = sc
    if 'cl_program' not in result['params']:
        result['params']['cl_program'] = 'none';
    result['power'] = result['energy'] # the name energy actually refers to "power".
    result['energy'] = result['time'] * result['power']
    return result

In [158]:
import dill as pickle
global_results = {}
load()

def make_key(k, u, result):
    arch = result['params']['arch']
    if arch == 'cpu':
        arch += '-1500'
    return '%d_%d_%s_%s_%s' %  (k, u, 
        arch, result['params']['cl_program'], result['params']['arithmetic'])

def exists(k, u, params):
    global global_results
    result = {
        'params': params
    }
    print make_key(k, u, result)
    if make_key(k, u, result) in global_results:
        return True
    return False
    
def save(k, u, result):
    global global_results
    global_results[make_key(k, u, result)] = result
    with open('result_conv.global', 'w') as f:
        pickle.dump(global_results, f)
        
def load():
    global global_results
    with open('result_conv.global', 'r') as f:
        global_results = pickle.load(f)


In [159]:
import itertools

In [160]:
global_results

{u'11_128_cpu-1500_none_float': {u'energy': 0.0101916662928,
  u'gflops': 10.6402,
  u'gflops_std': 0.0174733,
  u'params': {u'arch': u'cpu',
   u'arithmetic': u'float',
   'cl_program': 'none',
   u'sa': 128,
   u'sb': 363,
   u'sc': 3025},
  'power': 0.342248,
  u'time': 0.0297786,
  u'time_std': 0.0067509},
 u'11_128_gpu_blocking-2-v4_float': {u'energy': 0.009130475019600001,
  u'gflops': 6.77403,
  u'gflops_std': 0.0160881,
  u'params': {u'arch': u'gpu',
   u'arithmetic': u'float',
   u'cl_program': u'blocking-2-v4',
   u'sa': 128,
   u'sb': 363,
   u'sc': 3025},
  'power': 0.206877,
  u'time': 0.0441348,
  u'time_std': 0.000901419},
 u'11_256_cpu-1500_none_float': {u'energy': 0.014842851456,
  u'gflops': 15.619,
  u'gflops_std': 0.0164896,
  u'params': {u'arch': u'cpu',
   u'arithmetic': u'float',
   'cl_program': 'none',
   u'sa': 256,
   u'sb': 363,
   u'sc': 3025},
  'power': 0.386976,
  u'time': 0.038356,
  u'time_std': 0.00189277},
 u'11_256_gpu_blocking-2-v4_float': {u'energ

### conv1

In [164]:

align = 32
sc = 3025
d = 3
k_list = [5, 7, 9, 11, 13, 15]
u_list = [64, 96, 128, 256, 512]
num_iter = 100


    
for (k, u) in itertools.product(k_list, u_list):
    sa = u
    sb = k ** 2 * d

    def test_with_params(params):
        if not exists(k, u, params):
            result = test(align=align, sa=sa, sb=sb, sc=sc, iterations=num_iter, **params)
            print '\ttime = ', result['time'], 'gflops = ', result['gflops'], 'energy = ', result['energy']
            save(k, u, result)

    print 'gpu'
    test_with_params({
            'cl_program': 'blocking-2-v4',
            'arch': 'gpu',
            'arithmetic': 'float'
        })
    
    print 'cpu'
    test_with_params({
            'cl_program': 'none',
            'arch': 'cpu',
            'arithmetic': 'float'
        })


gpu
5_64_gpu_blocking-2-v4_float
cpu
5_64_cpu-1500_none_float
gpu
5_96_gpu_blocking-2-v4_float
cpu
5_96_cpu-1500_none_float
gpu
5_128_gpu_blocking-2-v4_float
cpu
5_128_cpu-1500_none_float
gpu
5_256_gpu_blocking-2-v4_float
cpu
5_256_cpu-1500_none_float
gpu
5_512_gpu_blocking-2-v4_float
cpu
5_512_cpu-1500_none_float
gpu
7_64_gpu_blocking-2-v4_float
cpu
7_64_cpu-1500_none_float
gpu
7_96_gpu_blocking-2-v4_float
cpu
7_96_cpu-1500_none_float
gpu
7_128_gpu_blocking-2-v4_float
cpu
7_128_cpu-1500_none_float
gpu
7_256_gpu_blocking-2-v4_float
cpu
7_256_cpu-1500_none_float
gpu
7_512_gpu_blocking-2-v4_float
cpu
7_512_cpu-1500_none_float
gpu
9_64_gpu_blocking-2-v4_float
cpu
9_64_cpu-1500_none_float
gpu
9_96_gpu_blocking-2-v4_float
cpu
9_96_cpu-1500_none_float
gpu
9_128_gpu_blocking-2-v4_float
cpu
9_128_cpu-1500_none_float
gpu
9_256_gpu_blocking-2-v4_float
cpu
9_256_cpu-1500_none_float
gpu
9_512_gpu_blocking-2-v4_float
cpu
9_512_cpu-1500_none_float
gpu
11_64_gpu_blocking-2-v4_float
cpu
11_64_cpu-1500

KeyboardInterrupt: 

In [165]:
# plot energy time tradeoff.
energy_list = []
time_list = []
name_list = []
for key in global_results:
    result = global_results[key]
    energy = [result['energy']]
    time = [result['time']]
    name_list.append(key)
    energy_list.append(energy)
    time_list.append(time)
    

In [169]:
plot_xy(time_list, energy_list, names=name_list, xlabel='time', ylabel='energy')

In [174]:
# plot energy time tradeoff, color by architecture.
energy_by_arch = {}
time_by_arch = {}
name_by_arch = {}
for key in global_results:
    result = global_results[key]
    arch = result['params']['arch']
    if arch not in energy_by_arch:
        energy_by_arch[arch] = []
        time_by_arch[arch] = []
        name_by_arch[arch] = []
    energy = result['energy']
    time = result['time']
    
    
    name_by_arch[arch].append(key)
    energy_by_arch[arch].append(energy)
    time_by_arch[arch].append(time)

archs = energy_by_arch.keys()

plot_xy([time_by_arch[key] for key in archs],
        [energy_by_arch[key] for key in archs],
        names=[name_by_arch[key] for key in archs])

In [188]:
# plot energy time tradeoff, color by conv filter.
traces = []
import numpy.random as npr

colors = {}
symbols = {
    'cpu-1500': 'cross',
    'gpu': 'circle'
}
for key in global_results:
    result = global_results[key]
    key_components = key.split('_')
    mat = 'x'.join(key_components[:2])
    arch = key_components[2]
    name = 'x'.join(key_components[:3])
    energy = result['energy']
    time = result['time']
    
    symbol = symbols.get(arch)
    if not symbol:
        continue
    
    color = colors.get(mat)
    if not color:
        color = 'rgba(%d, %d, %d, 1.)' % (npr.randint(0, 255), npr.randint(0, 255), npr.randint(0,255))
        colors[mat] = color
    
    
    
    trace = go.Scatter(
        x = [time],
        y = [energy],
        name = name,
        mode = 'markers',
        marker = dict(
            size = 10,
            color = color,
            symbol = symbol
        )
    )
    
    traces.append(trace)
    
    


layout = dict(title = 'time-energy tradeoff',
              yaxis = dict(zeroline = False, title='energy'),
              xaxis = dict(zeroline = False, title='time')
             )

fig = dict(data=traces, layout=layout)
iplot(fig)

In [184]:
traces

[]

In [186]:
global_results

{u'11_128_cpu-1500_none_float': {u'energy': 0.0101916662928,
  u'gflops': 10.6402,
  u'gflops_std': 0.0174733,
  u'params': {u'arch': u'cpu',
   u'arithmetic': u'float',
   'cl_program': 'none',
   u'sa': 128,
   u'sb': 363,
   u'sc': 3025},
  'power': 0.342248,
  u'time': 0.0297786,
  u'time_std': 0.0067509},
 u'11_128_gpu_blocking-2-v4_float': {u'energy': 0.009130475019600001,
  u'gflops': 6.77403,
  u'gflops_std': 0.0160881,
  u'params': {u'arch': u'gpu',
   u'arithmetic': u'float',
   u'cl_program': u'blocking-2-v4',
   u'sa': 128,
   u'sb': 363,
   u'sc': 3025},
  'power': 0.206877,
  u'time': 0.0441348,
  u'time_std': 0.000901419},
 u'11_256_cpu-1500_none_float': {u'energy': 0.014842851456,
  u'gflops': 15.619,
  u'gflops_std': 0.0164896,
  u'params': {u'arch': u'cpu',
   u'arithmetic': u'float',
   'cl_program': 'none',
   u'sa': 256,
   u'sb': 363,
   u'sc': 3025},
  'power': 0.386976,
  u'time': 0.038356,
  u'time_std': 0.00189277},
 u'11_256_gpu_blocking-2-v4_float': {u'energ